# GenAI-Camp: Day 03
## Lesson: Augmentation

This lesson is intended to show you how to enrich prompts with internal knowledge

During this lesson you will learn ...

- how to augment prompts with external knowledge
- about the disadvantages of uploading whole, large files


### Set up the environment
Import the necessary libraries, set constants, and define helper functions.

In [24]:
import os
from google import genai
from google.genai import types
from pathlib import Path

In [25]:
if os.getenv("COLAB_RELEASE_TAG"):
   from google.colab import userdata
   GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')
   COLAB = True
   print("Running on COLAB environment.")
else:
   from dotenv import load_dotenv, find_dotenv
   load_dotenv(find_dotenv())
   GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
   COLAB = False
   print("WARNING: Running on LOCAL environment.")
client = genai.Client(api_key=GOOGLE_API_KEY)

In [26]:
# Define path of ressources
if COLAB:
    # Clone the data repository into colab
    !git clone https://github.com/openknowledge/workshop-genai-camp-data.git
    DATA_PATH = "/content/workshop-genai-camp-data/day-03/data"
else:
    DATA_PATH = "../data"
BOOK_FILE = DATA_PATH + "/study_in_scarlett.txt"

In [27]:
# Set default values for model, model parameters and prompt
DEFAULT_MODEL = "gemini-2.0-flash"
DEFAULT_CONFIG_TEMPERATURE = 0.9
DEFAULT_CONFIG_TOP_K = 1
DEFAULT_CONFIG_MAX_OUTPUT_TOKENS = 200
DEFAULT_SYSTEM_PROMPT = "Your are a friendly assistant"
DEFAULT_USER_PROMPT = " "

# Define a function to generate completions using the Gemini model
def generate_gemini_completion(
        model_name: str = DEFAULT_MODEL, 
        temperature:float = DEFAULT_CONFIG_TEMPERATURE,
        top_k: int = DEFAULT_CONFIG_TOP_K, 
        max_output_tokens: int = DEFAULT_CONFIG_MAX_OUTPUT_TOKENS, 
        system_prompt : str = DEFAULT_SYSTEM_PROMPT, 
        user_prompt : str = DEFAULT_USER_PROMPT,
        txt_file: str | None = None,
        verbose: bool = False
        ) -> str: 
    
    """ Calls a gemini model with a given set of parameters and returns the completions 
    
    Parameters
    ----------
    model_name : str, optional [default: DEFAULT_GEMINI_MODEL]
        The name of the model to use for the completion
    temperature : float, optional [default: DEFAULT_CONFIG_TEMPERATURE]
        The temperature of the model
    top_k : int, optional [default: DEFAULT_CONFIG_TOP_K]
        The number of most recent matches to return
    max_output_tokens : int, optional [default: DEFAULT_CONFIG_MAX_OUTPUT_TOKENS]
        The maximum number of output tokens to return
    system_prompt : str, optional [default: DEFAULT_SYSTEM_PROMPT]
        The system prompt to use for the completion
    user_prompt : str, optional [default: DEFAULT_USER_PROMPT]
        The user prompt to use for the completion
    txt_file : str, optional [default: None]
        The path to a text file to use as input for the completion
        If None, the user_prompt is used instead
        If provided, the user_prompt is appended to the file content
    verbose : bool, optional [default: False]
        Whether to print details of the completion process or not. Defaults to False            
    Returns 
    -------
    str :
        the generated text      
    """    
    
    # create generation config 
    model_config = types.GenerateContentConfig(
        max_output_tokens=max_output_tokens,
        temperature=temperature,
        top_k=top_k,
        system_instruction=system_prompt,
    )

    # Append file content if provided
    contents = user_prompt
    if txt_file is not None:
        contents = [
            types.Part.from_bytes(
                data=Path(txt_file).read_bytes(),
                mime_type='text/plain',
            ),
            user_prompt
        ]
    
    # create generation request
    response = client.models.generate_content(
        model=model_name,
        contents=contents,
        config=model_config,
    )
    if verbose:
        print(f"Input tokens count: {response.usage_metadata.prompt_token_count}")
        print(f"Total tokens count: {response.usage_metadata.total_token_count}")
      
    
    return response.text

In [28]:
# Let's test the models knowledge on a book
user_input = "In Sherlock Holmes book 'A Study in Scarlett': Which number was written on the ceiling?"
generate_gemini_completion(user_prompt=user_input)

'In "A Study in Scarlet," the number written on the wall (not the ceiling) in blood at the crime scene is **RACHE**.\n'

### Exercise 01: Provide internal knowledge per file upload
In order to provide internal knowldge, gemini (and others) support uploading of documents. Your task is to provide the BOOK_FILE, representing internal knowledge alongside the prompt. Do you find a way of getting insights to token usage?  
**Hints**:
* See the implementation of the provided function for insights of token usage

In [29]:
# Question about a detailed part of the book
user_prompt = "Which number was written on the ceiling?"

# TODO: Call the gemini model with the user prompt and the book file
generate_gemini_completion(user_prompt=user_input, txt_file=BOOK_FILE, verbose=True)

Input tokens count: 62191
Total tokens count: 62203


'The number 28 was written on the ceiling.\n'

### Exercise 02: Enhance prompt with internal knowledge
Providing small pieces of context within the prompt should be the best way to answer this user query. Your task is to simply enhance the prompt below by providing the internal knowledge about the book.

In [30]:
# A paragraph from the book. which should be enough to answer the question
internal_book_knowledge_chunk = """
Still more shaken was he next morning. They had sat down to their
breakfast when Lucy with a cry of surprise pointed upwards. In the
centre of the ceiling was scrawled, with a burned stick apparently, the
number 28. To his daughter it was unintelligible, and he did not
enlighten her. That night he sat up with his gun and kept watch and
ward. He saw and he heard nothing, and yet in the morning a great 27
had been painted upon the outside of his door.
"""

user_prompt = "Which number was written on the ceiling?"

# TODO: Create a augmented prompt, where the user prompt is augmented with the internal book knowledge chunk
augmented_user_prompt = f"Answer the question below based on the context.\n\nContext: {internal_book_knowledge_chunk}\n\nQuestion: {user_prompt}"
generate_gemini_completion(user_prompt=augmented_user_prompt, verbose=True)

Input tokens count: 141
Total tokens count: 153


'The number 28 was written on the ceiling.\n'

### Exercise 03: Calculate costs for both calls
Costs is one factor, which should prevent us from uploading large file. But how much is it exactly? And how much more is it than just uploading small amounts of internal knowledge? Use only input token for calculation. Let's say we use the paid tier.  
**Hints**:  
* See [pricing](https://ai.google.dev/gemini-api/docs/pricing)

In [ ]:
# TODO: Calculate costs for input tokens for both calls and compare them
def calculate_costs(token_count: int, price_per_million_tokens: float = 1.25) -> float:
    """ Calculate the costs of a given number of tokens based on the price per million tokens """
    return token_count * (price_per_million_tokens / 1000000)

# Token counts for the two calls
input_token_count_file_upload = 62191
input_token_count_chunk_upload = 141

# For gemini-2.0-flash the price is 0.1 € per million tokens
euro_per_million_tokens = 0.1

costs_file_upload = calculate_costs(token_count=input_token_count_file_upload, price_per_million_tokens=euro_per_million_tokens)
costs_chunk_upload = calculate_costs(token_count=input_token_count_chunk_upload, price_per_million_tokens=euro_per_million_tokens)

print(f"Costs for {input_token_count_file_upload} tokens using file upload: {round(costs_file_upload, 5)} €")
print(f"Costs for {input_token_count_chunk_upload} tokens using chunk upload: {round(costs_chunk_upload, 5)} €")
print(f"Uploading the file is {round(costs_file_upload/costs_chunk_upload)} times more expensive than uploading the chunk.")

Costs for 62191 tokens using file upload: 0.00622 €
Costs for 141 tokens using chunk upload: 1e-05 €
Uploading the file is 441 times more expensive than uploading the chunk.
